In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
import os
from pathlib import Path
os.chdir('..')
DATA = Path('_data/genomics-data')

In [4]:
from SVMExperiments.utils import load_mat
import os

files = sorted(os.listdir(DATA/'DataCV/Raw'))
df = load_mat(DATA/f'DataCV/Raw/{files[0]}')

In [5]:
df.shape

(689862, 24)

In [5]:
df.head()

,CauseGene,EffectGene,Replicate,Treatment,Pvalue,0min_neg_cause,15min_cause,30min_cause,90min_cause,120min_cause,...,0min_neg_effect,15min_effect,30min_effect,90min_effect,120min_effect,180min_effect,210min_effect,240min_effect,360min_effect,Target
0,MAPK1,AGTRAP,1,BCR,0.008,1.570461,-0.259946,0.461187,-1.821926,0.217530,...,1.343140,0.967046,0.550347,-0.396333,0.762589,-0.857914,-0.271856,-1.741207,0.751194,0
1,MAPK1,AGTRAP,1,CD40,0.008,1.570461,0.540413,-0.058239,-0.384124,0.131689,...,1.343140,0.141576,0.677390,-1.477123,-1.339167,-1.487985,0.477539,-0.191964,-0.440323,0
2,MAPK1,AGTRAP,2,BCR,0.008,0.814767,-1.303725,0.780416,1.014104,0.493707,...,1.179496,1.347345,0.476586,-0.944036,-0.031326,-1.069956,-0.939858,-0.994739,-1.215449,0
3,MAPK1,AGTRAP,2,CD40,0.008,0.814767,-1.453817,0.690254,0.163129,0.340793,...,1.179496,0.093155,0.319289,-0.816700,-1.353020,0.005323,-0.141223,-0.072680,1.467384,0
4,MAPK1,AGTRAP,3,BCR,0.008,-1.276183,1.060551,0.513360,-0.863885,-0.520006,...,0.755433,1.313819,1.005074,-1.053026,-0.192625,-0.763318,-0.270895,-0.908896,0.380477,0


In [4]:
from scipy.io import loadmat
bags = sorted(os.listdir(DATA/'Bags'))
mat2 = loadmat(DATA/f'Bags/{bags[0]}')

In [5]:
genes = ['CTNNB1', 'Erk', 'Ikk2', 'IRF4', 'Jnk', 'LEF1', 'MYC']

In [7]:
from sklearn.metrics import confusion_matrix

In [8]:
def gmean_score(y, y_pred):
    matrix = confusion_matrix(y, y_pred)
    tn = matrix[0][0]
    tp = matrix[1][1]
    fn = matrix[1][0]
    fp = matrix[0][1]
    return np.sqrt(tp / (tp + fn) * tn / (tn + fp))

In [9]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer

In [11]:
X, y, cv_results = None, None, None
results = {}
for file in sorted(Path(DATA/'Bags').iterdir()):
    mat = loadmat(file)
    X = np.concatenate([
            mat2['NegGenePairsBag'], 
            mat2['PosGenePairsBag']
    ])
    y = np.concatenate([
        np.zeros(len(mat['NegGenePairsBag'])),
        np.ones(len(mat['PosGenePairsBag']))
    ])
    print(X.shape, y.shape)
    cv_results = cross_validate(LGBMClassifier(silent=False), X, y, cv=5, scoring=make_scorer(gmean_score))
    results[str(file.name)] = [np.mean(cv_results)['test_score'], np.std(cv_results)['test_score']]
    print(file.name, f'{100 * np.mean(cv_results)["test_score"]:.2f}', f'{100 * np.std(cv_results)["test_score"]:.2f}')
    break

(2000, 300) (2000,)


/home/raduviri/anaconda3/envs/chana/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/raduviri/anaconda3/envs/chana/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


KeyboardInterrupt: 

In [22]:
mat2 = loadmat(DATA/f'Bags/{bags[-1]}')

In [24]:
mat2['NegGenePairsBag'].shape

(1000, 300)